In [72]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns

In [73]:
d_train = pd.read_csv('train.csv')

In [74]:
d_train.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [75]:
d_test = pd.read_csv('test.csv')

In [76]:
d_test.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,101763,33.0,5.0,1.0,4.0,144.0,824.82,0.04,26.96,30.05,...,1257.60,30,0,3,0,21.0,23.0,87.0,57.0,9.0
1,101764,27.0,8.0,8.0,2.0,125.0,646.24,0.04,22.82,27.22,...,804.58,23,0,2,0,18.0,19.0,70.0,49.0,15.0
2,101765,130.0,11.0,7.0,10.0,545.0,3831.40,0.02,48.15,66.17,...,6453.34,99,9,17,1,26.0,53.0,333.0,244.0,21.0
3,101766,65.0,7.0,1.0,7.0,156.0,855.71,0.06,17.23,49.89,...,896.42,45,8,10,0,15.0,26.0,88.0,60.0,13.0
4,101767,22.0,3.0,1.0,3.0,52.0,238.42,0.10,9.60,26.70,...,145.80,16,0,4,0,12.0,15.0,30.0,24.0,5.0


In [77]:
d_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [78]:
d_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67842 entries, 0 to 67841
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67842 non-null  int64  
 1   loc                67842 non-null  float64
 2   v(g)               67842 non-null  float64
 3   ev(g)              67842 non-null  float64
 4   iv(g)              67842 non-null  float64
 5   n                  67842 non-null  float64
 6   v                  67842 non-null  float64
 7   l                  67842 non-null  float64
 8   d                  67842 non-null  float64
 9   i                  67842 non-null  float64
 10  e                  67842 non-null  float64
 11  b                  67842 non-null  float64
 12  t                  67842 non-null  float64
 13  lOCode             67842 non-null  int64  
 14  lOComment          67842 non-null  int64  
 15  lOBlank            67842 non-null  int64  
 16  locCodeAndComment  678

In [79]:
d_train.describe()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,...,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000
mean,50881.000000,37.347160,5.492684,2.845022,3.498826,96.655995,538.280956,0.111634,13.681881,27.573007,...,1141.357982,22.802453,1.773945,3.979865,0.196604,11.896131,15.596671,57.628116,39.249698,9.839549
std,29376.592059,54.600401,7.900855,4.631262,5.534541,171.147191,1270.791601,0.100096,14.121306,22.856742,...,9862.795472,38.541010,5.902412,6.382358,0.998906,6.749549,18.064261,104.537660,71.692309,14.412769
min,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,25440.500000,13.000000,2.000000,1.000000,1.000000,25.000000,97.670000,0.050000,5.600000,15.560000,...,31.380000,7.000000,0.000000,1.000000,0.000000,8.000000,7.000000,15.000000,10.000000,3.000000
50%,50881.000000,22.000000,3.000000,1.000000,2.000000,51.000000,232.790000,0.090000,9.820000,23.360000,...,125.400000,14.000000,0.000000,2.000000,0.000000,11.000000,12.000000,30.000000,20.000000,5.000000
75%,76321.500000,42.000000,6.000000,3.000000,4.000000,111.000000,560.250000,0.150000,18.000000,34.340000,...,565.920000,26.000000,1.000000,5.000000,0.000000,16.000000,20.000000,66.000000,45.000000,11.000000
max,101762.000000,3442.000000,404.000000,165.000000,402.000000,8441.000000,80843.080000,1.000000,418.200000,569.780000,...,935923.390000,2824.000000,344.000000,219.000000,43.000000,410.000000,1026.000000,5420.000000,3021.000000,503.000000


## we will do strandardization since the values are so different and some are big and some are very small

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

encoding the target variable to false as 0 and true as 1

In [81]:
def falseTrue(defects):
    if defects == True:
        return 1
    else:
        return 0

In [82]:
d_train['defects'] = d_train['defects'].apply(falseTrue)

In [83]:
d_train.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,0
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,0
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,0
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,1
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,0


separting the features and the target variable

In [84]:
X = d_train.drop(columns=['defects','id'],axis=1)
y = d_train['defects']

In [85]:
scaler.fit(X)

StandardScaler()

In [86]:
X_scaled = scaler.transform(X)

In [87]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.3,random_state=45)

In [88]:
model = XGBClassifier(objective='binary:logistic')

In [89]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001]
}

In [90]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(
    model,
    param_grid=param_grid,
    scoring='accuracy',  
    cv=5,  
    n_jobs=-1
)

In [91]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [92]:
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

In [93]:
best_params

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [94]:
best_estimator

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [95]:
model_tuned = XGBClassifier(objective='binary:logistic',**best_params)

In [96]:
model_tuned.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [97]:
y_pred = model_tuned.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81


In [98]:
ids = d_test['id']

In [99]:
d_test.drop(columns=['id'],axis=1,inplace=True)

In [100]:
d_test

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,33.0,5.0,1.0,4.0,144.0,824.82,0.04,26.96,30.05,22636.74,...,1257.60,30,0,3,0,21.0,23.0,87.0,57.0,9.0
1,27.0,8.0,8.0,2.0,125.0,646.24,0.04,22.82,27.22,14482.46,...,804.58,23,0,2,0,18.0,19.0,70.0,49.0,15.0
2,130.0,11.0,7.0,10.0,545.0,3831.40,0.02,48.15,66.17,116160.08,...,6453.34,99,9,17,1,26.0,53.0,333.0,244.0,21.0
3,65.0,7.0,1.0,7.0,156.0,855.71,0.06,17.23,49.89,16135.47,...,896.42,45,8,10,0,15.0,26.0,88.0,60.0,13.0
4,22.0,3.0,1.0,3.0,52.0,238.42,0.10,9.60,26.70,2624.49,...,145.80,16,0,4,0,12.0,15.0,30.0,24.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67837,41.0,1.0,1.0,1.0,125.0,656.55,0.07,14.00,47.61,8612.32,...,479.35,31,0,7,0,14.0,27.0,71.0,54.0,1.0
67838,20.0,3.0,1.0,3.0,38.0,161.42,0.15,6.75,23.28,1060.92,...,58.94,15,0,3,0,9.0,10.0,23.0,15.0,5.0
67839,24.0,2.0,1.0,2.0,52.0,240.00,0.11,9.00,26.67,2160.00,...,120.00,18,0,4,0,16.0,16.0,36.0,18.0,3.0
67840,18.0,2.0,1.0,1.0,49.0,216.64,0.11,9.33,24.02,2118.30,...,117.68,12,0,2,0,8.0,12.0,29.0,22.0,3.0


In [101]:
scaler.fit(d_test)

StandardScaler()

In [102]:
Test_scaler = scaler.transform(d_test)

In [103]:
Test_scaler

array([[-0.08703423, -0.06791786, -0.39810657, ...,  0.27182298,
         0.23578527, -0.06425941],
       [-0.19457362,  0.29627789,  1.10199993, ...,  0.11067129,
         0.12582837,  0.32680193],
       [ 1.65151933,  0.66047364,  0.887699  , ...,  2.60378279,
         2.80602766,  0.71786327],
       ...,
       [-0.24834332, -0.4321136 , -0.39810657, ..., -0.2116321 ,
        -0.30025459, -0.45532075],
       [-0.35588271, -0.4321136 , -0.39810657, ..., -0.27798868,
        -0.24527614, -0.45532075],
       [ 1.13174558,  1.14606796,  1.53060179, ..., -0.55289451,
        -0.5476576 ,  1.23927839]])

In [104]:
predictions = model_tuned.predict(Test_scaler)

# Create a DataFrame with 'ID' and 'Defect' columns
result_df = pd.DataFrame({'id': ids, 'defects': predictions})

In [105]:
result_df.head()

,id,defects
0,101763,0
1,101764,0
2,101765,1
3,101766,0
4,101767,0


In [106]:
result_df.to_csv("submission.csv",index=False)

In [107]:
len(predictions)

67842

In [108]:
import pickle
with open('tuned_model1.pkl', 'wb') as file:
    pickle.dump(model_tuned, file)

In [109]:
from sklearn.metrics import roc_auc_score

# Assuming you have a tuned XGBoost model 'model_tuned' and test data 'X_test' and 'y_test'
y_pred_proba = model_tuned.predict_proba(x_test)[:, 1]  # Get the predicted probabilities for the positive class

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC: {auc:.2f}")


AUC: 0.79


###### new 

X_scaled,y

In [110]:
model2 = XGBClassifier(objective='binary:logistic',**best_params)

In [111]:
model2.fit(X_scaled,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [112]:
predictions = model2.predict(Test_scaler)

# Create a DataFrame with 'ID' and 'Defect' columns
result_df1 = pd.DataFrame({'id': ids, 'defects': predictions})

In [113]:
result_df1.head()

,id,defects
0,101763,0
1,101764,0
2,101765,1
3,101766,0
4,101767,0


In [114]:
result_df1.to_csv("new_sub2.csv",index=False)

In [115]:
d_train.columns

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')

##### deep learning model

In [116]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [117]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
2227/2227 [==============================] - 2s 628us/step - loss: 0.4411 - accuracy: 0.8074 - val_loss: 0.4415 - val_accuracy: 0.8107
Epoch 2/10
2227/2227 [==============================] - 1s 590us/step - loss: 0.4333 - accuracy: 0.8135 - val_loss: 0.4329 - val_accuracy: 0.8141
Epoch 3/10
2227/2227 [==============================] - 1s 591us/step - loss: 0.4320 - accuracy: 0.8140 - val_loss: 0.4365 - val_accuracy: 0.8124
Epoch 4/10
2227/2227 [==============================] - 1s 590us/step - loss: 0.4311 - accuracy: 0.8134 - val_loss: 0.4327 - val_accuracy: 0.8132
Epoch 5/10
2227/2227 [==============================] - 1s 588us/step - loss: 0.4307 - accuracy: 0.8140 - val_loss: 0.4326 - val_accuracy: 0.8143
Epoch 6/10
2227/2227 [==============================] - 1s 592us/step - loss: 0.4303 - accuracy: 0.8146 - val_loss: 0.4329 - val_accuracy: 0.8147
Epoch 7/10
2227/2227 [==============================] - 1s 589us/step - loss: 0.4301 - accuracy: 0.8152 - val_loss: 0.4327 -

In [118]:
from sklearn.metrics import roc_auc_score

y_pred_proba = model.predict(x_test)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC Score: {auc:.2f}")

955/955 [==============================] - 0s 276us/step
AUC Score: 0.79


In [119]:
y_predictions1 = model.predict(Test_scaler)
# Assuming y_predictions1 is a 2D array, flatten it to 1D
y_predictions1 = y_predictions1.flatten()

# Create a DataFrame with 'id' and 'defects' columns
result_df2 = pd.DataFrame({'id': ids, 'defects': y_predictions1})

2121/2121 [==============================] - 1s 268us/step


In [120]:
result_df2.to_csv("deepL.csv",index=False)

###### model dl 2

In [121]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dropout(0.5),  # Dropout layer
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model and calculate AUC
y_pred_proba = model.predict(x_test)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC Score: {auc:.2f}")


Epoch 1/20
2227/2227 [==============================] - 2s 742us/step - loss: 0.5377 - accuracy: 0.8000 - val_loss: 0.4546 - val_accuracy: 0.8018
Epoch 2/20
2227/2227 [==============================] - 2s 711us/step - loss: 0.4588 - accuracy: 0.8048 - val_loss: 0.4456 - val_accuracy: 0.8124
Epoch 3/20
2227/2227 [==============================] - 2s 708us/step - loss: 0.4572 - accuracy: 0.8052 - val_loss: 0.4440 - val_accuracy: 0.8123
Epoch 4/20
2227/2227 [==============================] - 2s 707us/step - loss: 0.4558 - accuracy: 0.8027 - val_loss: 0.4464 - val_accuracy: 0.8147
Epoch 5/20
2227/2227 [==============================] - 2s 710us/step - loss: 0.4556 - accuracy: 0.8031 - val_loss: 0.4450 - val_accuracy: 0.8136
Epoch 6/20
2227/2227 [==============================] - 2s 710us/step - loss: 0.4543 - accuracy: 0.8051 - val_loss: 0.4448 - val_accuracy: 0.8143
Epoch 7/20
2227/2227 [==============================] - 2s 714us/step - loss: 0.4544 - accuracy: 0.8028 - val_loss: 0.4434 -

In [122]:
y_predictions2 = model.predict(Test_scaler)
y_predictions2 = y_predictions2.flatten()
result_df3 = pd.DataFrame({'id': ids, 'defects': y_predictions2})

2121/2121 [==============================] - 1s 286us/step


In [123]:
result_df3.to_csv("deepL2.csv",index=False)

## doing the min-max scaling in kaggle

In [132]:
X1 = d_train.drop(columns=['id','defects'])
y1 = d_train['defects']

In [133]:
from sklearn.preprocessing import MinMaxScaler

# Create an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on your data and transform it
X_scaled1 = scaler.fit_transform(X1)

In [134]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled1,y1,random_state=355,test_size=0.30)

In [135]:
rf = RandomForestClassifier(random_state=42)

In [136]:
grid_param = {
    'n_estimators':[2,70,100,120],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features':['sqrt','log2']
}

In [137]:
grid_search = GridSearchCV(estimator=rf,cv=5,param_grid=grid_param,n_jobs=-1,verbose=3)

In [138]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.794 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.806 total time=   1.7s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=120;, score=0.806 total time=   2.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=100;, score=0.804 total time=   1.6s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=2;, score=0.797 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=2;, score=0.795 total time=   0.0s
[CV 5/5] END criterion=gin

[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=70;, score=0.806 total time=   1.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.810 total time=   1.6s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=70;, score=0.805 total time=   1.2s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=120;, score=0.806 total time=   2.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=70;, score=0.806 total time=   1.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=120;, score=0.811 total time=   2.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_sample

[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=70;, score=0.811 total time=   1.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=120;, score=0.811 total time=   2.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=100;, score=0.806 total time=   1.6s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=120;, score=0.806 total time=   2.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=100;, score=0.810 total time=   1.8s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=70;, score=0.806 total time=   1.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_sampl

[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.793 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.804 total time=   1.7s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=120;, score=0.810 total time=   2.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=100;, score=0.806 total time=   1.6s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=2;, score=0.793 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=2;, score=0.790 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_s

[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.797 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.811 total time=   1.7s
[CV 1/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=2;, score=0.793 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=2;, score=0.797 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=2;, score=0.790 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=3, n_estimators=2;, score=0.795 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split

...

[CV 3/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=8, n_estimators=100;, score=0.811 total time=   3.7s
[CV 1/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=9, n_estimators=2;, score=0.810 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=9, n_estimators=2;, score=0.810 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=9, n_estimators=2;, score=0.812 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=9, n_estimators=70;, score=0.817 total time=   2.6s
[CV 1/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=9, n_estimators=100;, score=0.812 total time=   3.8s
[CV 4/5] END criterion=gini, max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_sp

[CV 2/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=70;, score=0.815 total time=   2.9s
[CV 1/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=100;, score=0.812 total time=   4.1s
[CV 4/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=120;, score=0.810 total time=   5.0s
[CV 2/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=8, n_estimators=100;, score=0.816 total time=   4.3s
[CV 1/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=9, n_estimators=2;, score=0.807 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_split=9, n_estimators=2;, score=0.804 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=1, min_samples_

[CV 3/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=2;, score=0.806 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=2;, score=0.808 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=70;, score=0.812 total time=   2.8s
[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=70;, score=0.815 total time=   2.7s
[CV 1/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=120;, score=0.812 total time=   4.7s
[CV 4/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=7, n_estimators=70;, score=0.810 total time=   2.7s
[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_sp

[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=9, n_estimators=2;, score=0.811 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=9, n_estimators=70;, score=0.816 total time=   2.7s
[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=9, n_estimators=70;, score=0.816 total time=   2.7s
[CV 1/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=9, n_estimators=120;, score=0.812 total time=   4.8s
[CV 4/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=70;, score=0.810 total time=   2.8s
[CV 5/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=100;, score=0.815 total time=   3.9s
[CV 3/5] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_leaf=3, min_samples_

KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

In [ ]:
rf_tuned = RandomForestClassifier(**grid_search.best_params_)